# Code to setup the blocks for a 4D Multigrid run
March 2, 2022

In [1]:
import numpy as np


In [2]:
keys=['lattice','node_geo','size_per_gpu','nodes']

In [3]:
def f_to_numpy(dict1,key_lst):
    for key in key_lst:
        dict1[key]=np.array(dict1[key],dtype=np.int16)
        
def f_divide_arrays(a1,a2):
    a3=a1//a2
    a4=a1%a2
    
    if np.any(a4):
        print("Found non-zero remainder. Integer division is not correct")
        print("Remainder list",a4)
    return a3

### Fixed entries

In [4]:
# dict_full={'lattice':[144, 144, 144, 288]}
# dict_tune={'node_geo':[6,2,2,2],'nodes':8}

dict_full={'lattice':[96, 96, 96, 192]}
dict_tune={'node_geo':[2, 2, 2, 4],'nodes':4}

### Varying entries 

In [5]:
# dict_full['node_geo']=[6,3,6,8]
# dict_full['nodes']=144
# dict_blocks={'b1':[4,6,6,6], 'b2':[3,2,2,3]}

dict_full['node_geo']=[4, 4, 2, 4]
prd=1
for i in dict_full['node_geo']: prd*=i 
dict_full['nodes']=prd/8
dict_blocks={'b1':[4,4,4,6], 'b2':[3,3,3,2]}

### Compute

In [6]:
# Convert lists to numpy arrays
f_to_numpy(dict_full,['lattice','node_geo'])
f_to_numpy(dict_tune,['node_geo'])

# Compute size per gpu for full run
dict_full['size_per_gpu']=f_divide_arrays(dict_full['lattice'],dict_full['node_geo'])

# Compute lattice size for tuning run
dict_tune['size_per_gpu']=dict_full['size_per_gpu']
dict_tune['lattice']=dict_tune['size_per_gpu']*dict_tune['node_geo']


# Compute block sizes inside GPU
f_to_numpy(dict_blocks,['b1','b2'])
loc_vol1=f_divide_arrays(dict_tune['size_per_gpu'],dict_blocks['b1'])
loc_vol2=f_divide_arrays(loc_vol1,dict_blocks['b2'])

### Print output

In [7]:
print("Size   ",dict_tune['size_per_gpu'])
print("Block 1",dict_blocks['b1'],"->",loc_vol1)
print("\nSize   ",loc_vol1)
print("Block 2",dict_blocks['b2'],"->",loc_vol2)


Size    [24 24 48 48]
Block 1 [4 4 4 6] -> [ 6  6 12  8]

Size    [ 6  6 12  8]
Block 2 [3 3 3 2] -> [2 2 4 4]


In [8]:
## Print run paramters
print("{:<16} {:<15} {:<10}".format(' ','Tuning run','Full run'))
for k in keys:
    v1,v2=dict_tune[k],dict_full[k]
    if k!='nodes':
        w1=" ".join([str(i) for i in v1])
        w2=" ".join([str(i) for i in v2])
        print("{:<16} {:<15} {:<10}".format(k,w1,w2))
    else :
        print("{:<16} {:<15} {:<10}".format(k,str(v1),str(v2)))


                 Tuning run      Full run  
lattice          48 48 96 192    96 96 96 192
node_geo         2 2 2 4         4 4 2 4   
size_per_gpu     24 24 48 48     24 24 48 48
nodes            4               16.0      
